In [3]:
import sys
sys.path.append('../Custom Library/') 
import datetime as dt
import HR_Analytics as hr
import pandas as pd

In [4]:
engine = hr.setEngine('credentials','BIHRDW')

### Quarterly Attrition rates for all tracked ethnic groups in US from 2018 through 1H 21. (Exempt Only) 

In [3]:
attritQuery = """
Declare @FYStart As Date = '2017-10-31'

Select	
	HWD.[Report Date],
	Case
		When HWD.[Management Level Category] <> N'NONE' Then 'Yes'
		Else 'No'
	End As [Is ProfessiOnal],
	Case
		When HWD.[Original Hire Date] > @FYStart Or HWD.[Hire Date] > @FYStart Then 'Yes'
		Else 'No'
	End As [Is New Hire],
	HWD.[Management Level Category],
	count(1) As [HC],
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End As [Gender Code],
    HWD.[Ethnicity]
From 
	HP_WORKER_DATA As HWD
Where 
	HWD.[Report Date] > @FYStart and HWD.[Report Date] <= '2018-10-31' And 
	[Worker Reg / Temp Code] = N'R' And 
	[Worker Status CategOry Code] = N'A'  And
    HWD.Exempt Like 'Y'
    And
    HWD.[Pay Group Country Desc] Like 'United States of America'
Group By
HWD.[Report Date], HWD.[Management Level Category], Case
		When HWD.[Original Hire Date] > @FYStart Or HWD.[Hire Date] > @FYStart Then 'Yes'
		Else 'No'
	End,   Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End, HWD.[Ethnicity]

Declare @FYStart As Date = '2017-10-31'

Select	
	Eomonth(dateadd(d, 1, HWD.[TerminatiOn Date]), 0) As [Report Date],
	Case
		When HWD.[Management Level Category] <> N'NONE' Then 'Yes'
		Else 'No'
	End As [Is ProfessiOnal],
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End As [Gender Code],
    HWD.[Ethnicity],
	Case
		When TT.[Job Family Code] is null Then 'No'
	Else 'Yes'
	End As [Technical Job Family],
    hwd.age,
	0 As [HC],
	count(1) AS [Voluntary Attrits]
From HP_WORKER_DATA As HWD

Inner Join HP_ATTRITION As TD On TD.[Report Date] = HWD.[Report Date]
	And TD.[WOrker ID] = HWD.[Worker ID]
	And HWD.[AttritiOn Type] = 'Voluntary'
	
--Inner Join @HPI_Orgs hp On hp.[Business Lvl 1 (Group) Code] = wd.[Business Lvl 1 (Group) Code]
Left Join TECH_JOBS_FAM As TT On TT.[Job Family Code] = HWD.[Job Family Code]
--Left Join TechnicalJobs tj On tj.[Job Family] = wd.[Job Family]
Where 
	EomOnth(dateadd(d, 1, HWD.[TerminatiOn Date]), 0) > @FYStart  and HWD.[Report Date] <= '2018-10-31' 
    And HWD.[Pay Group Country Code] = N'USA'
    and hwd.Exempt Like 'Y'
Group by 
	EomOnth(dateadd(d, 1, HWD.[TerminatiOn Date]), 0),
    Case
		When HWD.[Management Level Category] <> N'NONE' Then 'Yes'
		Else 'No'
	End,
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End,
	Case
		When TT.[Job Family Code] is null Then 'No'
	Else 'Yes'
	End,
    HWD.[Ethnicity],
    hwd.age
"""

In [4]:
attrition = hr.internalQuery(attritQuery,engine)

In [5]:
attrition = attrition.drop_duplicates(subset=['Worker ID'])

In [19]:
# Pivot Data and Fill NA with 0
pivAttrit = attrition.pivot_table(index='Report Date', columns='Ethnicity', values='Worker ID', aggfunc=pd.Series.nunique).fillna(0)

toExport = pivAttrit.drop(columns=['Unknown','Not Specified (United States of America)','I choose not to disclose (United States of America)'])
hr.exportData(toExport,'attrition_quarterly','Data')
pivAttrit.drop(columns='Unknown')

Ethnicity,American Indian/Alaska Native (United States of America),Asian (United States of America),Black/African American (United States of America),Hispanic/Latino (United States of America),I choose not to disclose (United States of America),Native Hawaiian/Oth Pac Island (United States of America),Not Specified (United States of America),Two or More Races (United States of America),White (United States of America)
Report Date,,,,,,,,,
2018-10-31,2.0,75.0,36.0,57.0,0.0,2.0,19.0,18.0,325.0
2019-09-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
2019-10-31,6.0,92.0,32.0,60.0,13.0,1.0,0.0,15.0,388.0
2020-09-30,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,8.0
2020-10-31,3.0,69.0,17.0,42.0,16.0,2.0,0.0,17.0,307.0
2021-03-31,0.0,20.0,11.0,11.0,8.0,0.0,0.0,2.0,70.0
2021-04-30,1.0,16.0,2.0,9.0,2.0,1.0,0.0,7.0,89.0


### Quarterly Hire rates for all tracked ethnic groups in US from 2018 through 1H 21. (Exempt Only) 

In [9]:
hireQuery = """
Declare @FYStart As Date = '2020-10-31'


Select	
	HWD.[Report Date],
	Case
		When HWD.[Management Level Category] <> N'NONE' Then 'Yes'
		Else 'No'
	End As [Is ProfessiOnal],
	Case
		When HWD.[Original Hire Date] > @FYStart Or HWD.[Hire Date] > @FYStart Then 'Yes'
		Else 'No'
	End As [Is New Hire],
	HWD.[Management Level Category],
	count(1) As [HC],
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End As [Gender Code],
    HWD.[Ethnicity]
From 
	HP_WORKER_DATA As HWD
Where 
	HWD.[Report Date] > '2017-10-31' And 
	[Worker Reg / Temp Code] = N'R' And 
	[Worker Status CategOry Code] = N'A'  And
    HWD.Exempt Like 'Y" 
    And
    HWD.[Pay Group Country Desc] Like 'United States of America' 
"""

In [10]:
hire = hr.internalQuery(hireQuery,engine)

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near 'United'. (102) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Unclosed quotation mark after the character string ' \n'. (105)")
[SQL: 
Declare @FYStart As Date = '2020-10-31'


Select	
	HWD.[Report Date],
	Case
		When HWD.[Management Level Category] <> N'NONE' Then 'Yes'
		Else 'No'
	End As [Is ProfessiOnal],
	Case
		When HWD.[Original Hire Date] > @FYStart Or HWD.[Hire Date] > @FYStart Then 'Yes'
		Else 'No'
	End As [Is New Hire],
	HWD.[Management Level Category],
	count(1) As [HC],
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End As [Gender Code],
    HWD.[Ethnicity]
From 
	HP_WORKER_DATA As HWD
Where 
	HWD.[Report Date] > '2017-10-31' And 
	[Worker Reg / Temp Code] = N'R' And 
	[Worker Status CategOry Code] = N'A'  And
    HWD.Exempt Like 'Y" 
    And
    HWD.[Pay Group C

In [9]:
hire = hire.drop_duplicates(subset=['Worker ID'])

In [20]:
pivHire= hire.pivot_table(index='Report Date', columns='Ethnicity', values='Worker ID', aggfunc=pd.Series.nunique).fillna(0)

toExport = pivHire.drop(columns=['Unknown','Not Specified (United States of America)','I choose not to disclose (United States of America)'])
hr.exportData(toExport,'hire_quarterly','Data')
pivHire.drop(columns='Unknown')

Ethnicity,American Indian/Alaska Native (United States of America),Asian (United States of America),Black/African American (United States of America),Chinese (Malaysia),Hispanic/Latino (United States of America),I choose not to disclose (United States of America),Native Hawaiian/Oth Pac Island (United States of America),Not Specified (United States of America),Two or More Races (United States of America),White (United States of America)
Report Date,,,,,,,,,,
2018-01-31,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0
2018-02-28,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
2018-03-31,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2018-04-30,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
2018-05-31,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0
2018-06-30,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0
2018-07-31,0.0,3.0,0.0,0.0,5.0,0.0,0.0,1.0,1.0,7.0
2018-08-31,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0
2018-09-30,0.0,4.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,16.0


### Quarterly Promotion rates for all tracked ethnic groups in US from 2018 through 1H21. (Exempt Only) 

In [11]:
promoQuery = """
Select  Distinct  
    HWD1.[Report Date],
	HWD1.[Worker ID],
    Case
    When 
        HWD1.[Gender Code] Is Null Then 'U'
    Else
        HWD1.[Gender Code]
    End As [Gender Code],
    HWD1.[Ethnicity]
From 
	HC_TRACK_CHANGES As HTC
	Inner Join JOB_MOVES As JM On JM.[Move Type] = HTC.[ChangeSubCategory]
	Left Join HP_WORKER_DATA As HWD1 On HWD1.[Report Date] = HTC.[ReportDate] And HWD1.[Worker ID] = HTC.EmployeeID
	Left Join HP_WORKER_DATA As HWD2 On HWD2.[Report Date] = HTC.[PreviousReportDate] And HWD2.[Worker ID] = HTC.EmployeeID
Where 
	HTC.ReportDate >= '2018-01-01'
And     
	HTC.[ChangeDescription] Like '%Promotion%'
And
    HWD1.[Pay Group Country Desc] Like 'United States of America'
And
    HWD1.[Exempt] Like 'Y'
"""

In [12]:
promotion = hr.internalQuery(promoQuery,engine)

In [13]:
promotion = promotion.drop_duplicates(subset=['Worker ID'])

In [21]:
pivPromo= promotion.pivot_table(index='Report Date', columns='Ethnicity', values='Worker ID', aggfunc=pd.Series.nunique).fillna(0)

toExport = pivPromo.drop(columns=['Unknown','Not Specified (United States of America)','I choose not to disclose (United States of America)'])
hr.exportData(toExport,'promotion_quarterly','Data')
pivPromo.drop(columns='Unknown')

Ethnicity,American Indian/Alaska Native (United States of America),Asian (United States of America),Black/African American (United States of America),Hispanic/Latino (United States of America),I choose not to disclose (United States of America),Native Hawaiian/Oth Pac Island (United States of America),Not Specified (United States of America),Two or More Races (United States of America),White (United States of America)
Report Date,,,,,,,,,
2018-01-31,0.0,7.0,4.0,3.0,0.0,0.0,0.0,1.0,35.0
2018-02-28,0.0,14.0,2.0,9.0,0.0,0.0,2.0,1.0,28.0
2018-03-31,0.0,6.0,1.0,2.0,0.0,0.0,2.0,2.0,39.0
2018-04-30,0.0,6.0,0.0,2.0,0.0,0.0,1.0,2.0,16.0
2018-05-31,2.0,48.0,9.0,24.0,0.0,0.0,10.0,5.0,164.0
2018-06-30,0.0,3.0,2.0,11.0,0.0,0.0,1.0,2.0,39.0
2018-07-31,0.0,8.0,1.0,4.0,0.0,0.0,2.0,0.0,38.0
2018-08-31,1.0,6.0,2.0,7.0,0.0,0.0,5.0,2.0,32.0
2018-09-30,2.0,8.0,0.0,3.0,0.0,0.0,3.0,2.0,26.0


Quarterly Tenure of Attritted exempt employees across all ethnic groups in US from 2018 through 1H21.

In [22]:
attritQuery = """
Select Distinct
    HWD.[Report Date],
    HWD.[Termination Date],
	HWD.[Worker ID],
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End As [Gender Code],
    HWD.[Length of Service in Years],
    HWD.[Ethnicity]
From 
	HP_WORKER_DATA As HWD
Inner Join 
    HP_ATTRITION AS TD On TD.[Report Date] = HWD.[Report Date] 
And 
    HWD.[Worker ID] = TD.[Worker ID]
And 
    HWD.[Attrition Type] = 'Voluntary'
Where 
	HWD.[Termination Date] >='2018-01-01'
And 
    HWD.[Job Family] Not Like 'College'
And
    HWD.[Pay Group Country Desc] Like 'United States of America' 
"""

In [23]:
attritTen = hr.internalQuery(attritQuery,engine)

In [26]:
attritTen = attritTen.drop_duplicates(subset=['Worker ID'])
attritTen

,Report Date,Termination Date,Worker ID,Gender Code,Length of Service in Years,Ethnicity
0,2018-10-31,2018-01-02,10023079,M,13,White (United States of America)
1,2018-10-31,2018-01-02,21409411,M,7,Hispanic/Latino (United States of America)
2,2018-10-31,2018-01-03,00170038,F,20,Asian (United States of America)
3,2018-10-31,2018-01-03,90005365,M,1,Unknown
4,2018-10-31,2018-01-04,60029358,M,3,Black/African American (United States of America)
...,...,...,...,...,...,...
2207,2020-10-31,2020-09-30,07847316,M,18,White (United States of America)
2208,2020-10-31,2020-10-18,90027279,M,1,Asian (United States of America)
2263,2021-04-30,2021-04-02,90009388,M,4,White (United States of America)
2264,2021-04-30,2021-04-16,90017851,M,2,Two or More Races (United States of America)


In [28]:
pivattritTen= attritTen.pivot_table(index='Report Date', columns='Ethnicity', values='Worker ID', aggfunc=pd.Series.nunique).fillna(0)

toExport = pivattritTen.drop(columns=['Unknown','Not Specified (United States of America)','I choose not to disclose (United States of America)'])
hr.exportData(attritTen,'attrition_tenure_quarterly','Data')

In [5]:
attritQuery = """
Select Distinct
    HWD.[Report Date],
    HWD.[Termination Date],
	HWD.[Worker ID],
    Case
    When 
        HWD.[Gender Code] Is Null Then 'U'
    Else
        HWD.[Gender Code]
    End As [Gender Code],
    HWD.[Age],
    HWD.[Ethnicity],
    HWD.[Job Family]
From 
	HP_WORKER_DATA As HWD
Inner Join 
    HP_ATTRITION AS TD On TD.[Report Date] = HWD.[Report Date] 
And 
    HWD.[Worker ID] = TD.[Worker ID]
And 
    HWD.[Attrition Type] = 'Voluntary'
Where 
	HWD.[Termination Date] >='2018-01-01'
And 
    HWD.[Job Family] Not Like 'College'
And
    HWD.[Pay Group Country Desc] Like 'United States of America'
And
    HWD.Exempt Like 'Y'  
"""

In [6]:
attritTen = hr.internalQuery(attritQuery,engine)

In [7]:
pivattritTen= attritTen.pivot_table(index='Report Date', columns='Ethnicity', values='Worker ID', aggfunc=pd.Series.nunique).fillna(0)

toExport = pivattritTen.drop(columns=['Unknown','Not Specified (United States of America)','I choose not to disclose (United States of America)'])
hr.exportData(attritTen,'attrition_jf_quarterly','Data')